### 0. Import Libraries

In [6]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

### 1.1 Download Dataset

In [4]:
X, y = fetch_openml('mnist_784', version=1, return_X_y=True)

### 1.2 Train/Test split and dataset normalization

In [14]:
permutation = np.random.permutation(X.shape[0])
X = X[permutation]
y = y[permutation]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = X_train.reshape((X_train.shape[0], 28, 28))
X_test = X_test.reshape((X_test.shape[0], 28, 28))

### 1.3 Hyperparameters

In [18]:
inputs=28 + 1 # bias input
hidden=30
outputs=10

batch_size=10
epochs = 1500
lr = 0.001

### 2.1 MNN Functions (check Scikit 2D notebook for details)

In [17]:
def f(x, derivative=False):
    gt = (x > 0)
    if derivative:
        return 1 * gt
    else:
        return x * gt

def softmax(x):
    exps = np.exp(x - np.max(x))
    return exps / np.expand_dims(np.sum(exps, axis=1), axis=1)


def ce_loss(out, y, grad):
    y = y.astype(int)
    m = y.shape[0]
    p = softmax(out)
    log_likelihood = -np.log(p[range(m),y])
    loss = np.mean(log_likelihood, axis=0)

    de = p
    de[range(m),y] -= 1
    de = de/m
    de = np.expand_dims(np.expand_dims(de, axis=2), axis=2)
    grad = de*grad

    return (loss, grad.sum(axis=1).sum(axis=0))

def derivate(grad, t):
    sn = np.zeros(shape=grad.shape)
    sn[:,np.eye(neurons).astype(bool)] = state[:,np.newaxis]
    sn = np.transpose(sn, (0,3,2,1))
    return f(t, derivative=True)[:, np.newaxis, np.newaxis] * (np.matmul(grad, A) + sn)

def net(state, grad, x, bs=batch_size, compute_grad=True):
    # set inputs neurons state with input vector
    state[:,0:inputs] = np.concatenate((x, np.ones((bs,1))), axis=1)
    
    # compute ti
    t = np.matmul(state, A)
    
    # Forward propagate the gradients
    grad = derivate(grad, t) if compute_grad else None
    
    # Compute new state
    state = f(t)
    
    return state, grad

### 2.2 Adam Optimizer

In [16]:
class Adam:
    def __init__(self, lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0., **kwargs):
        
        allowed_kwargs = {'clipnorm', 'clipvalue'}
        for k in kwargs:
            if k not in allowed_kwargs:
                raise TypeError('Unexpected keyword argument '
                                'passed to optimizer: ' + str(k))

        self.__dict__.update(kwargs)
        self.iterations = 0
        self.lr = lr
        self.beta_1 = beta_1
        self.beta_2 = beta_2
        self.decay = decay
        self.epsilon = epsilon
        self.initial_decay = decay

    def step(self, params, grads):
        original_shapes = [x.shape for x in params]
        params = [x.flatten() for x in params]
        grads = [x.flatten() for x in grads]
        

        lr = self.lr
        if self.initial_decay > 0:
            lr *= (1. / (1. + self.decay * self.iterations))

        t = self.iterations + 1
        lr_t = lr * (np.sqrt(1. - np.power(self.beta_2, t)) /
                     (1. - np.power(self.beta_1, t)))

        if not hasattr(self, 'ms'):
            self.ms = [np.zeros(p.shape) for p in params]
            self.vs = [np.zeros(p.shape) for p in params]
    
        ret = [None] * len(params)
        for i, p, g, m, v in zip(range(len(params)), params, grads, self.ms, self.vs):
            m_t = (self.beta_1 * m) + (1. - self.beta_1) * g
            v_t = (self.beta_2 * v) + (1. - self.beta_2) * np.square(g)
            p_t = p - lr_t * m_t / (np.sqrt(v_t) + self.epsilon)
            self.ms[i] = m_t
            self.vs[i] = v_t
            ret[i] = p_t
        
        self.iterations += 1
        
        for i in range(len(ret)):
            ret[i] = ret[i].reshape(original_shapes[i])
        
        return ret

In [ ]:
neurons = inputs + hidden + outputs
A = np.random.rand(neurons, neurons)
optimizer = Adam(lr=lr)

for epoch in range(0, epochs):
    losses = 0
    for i in range(0, len(X_train)//batch_size):
        # Training batches
        batch_X = X_train[i*batch_size:(i+1)*batch_size]
        batch_Y = y_train[i*batch_size:(i+1)*batch_size]
        
        state = np.zeros(shape=(batch_size, neurons)) # Init state
        grad = np.zeros(shape=(batch_size, neurons, neurons, neurons)) # Init gradients

        # Update MNN in time
        for t in range(0, 28):
            state, grad = net(state, grad, batch_X[:, t, :])
        
        # Permute gradients for error function
        grad = np.transpose(grad, (0,3,1,2))
        
        # Slice output values and gradients
        outs = state[:,inputs+hidden:neurons]
        outs_grad = grad[:,inputs+hidden:neurons]
        
        # Compute loss and error gradients
        loss, err_grad = ce_loss(outs, batch_Y, outs_grad)
        losses += loss

        # Update weights
        A = optimizer.step(A, err_grad)
        
        print("epoch: %d  loss: %f" % (epoch, losses/(len(train_X)//(batch_size))))
